In [1]:
from torch.utils.data import DataLoader
import pandas as pd
import pickle
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer
from sentence_transformers import losses

In [2]:
# Downgrade PyTorch (fixes the frozen dataclass issue)
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2

# Install compatible versions for SBERT fine-tuning
!pip install -U sentence-transformers==2.2.2 transformers==4.31.0 huggingface_hub==0.15.1

ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_lp64.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_intel_thread.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_def.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_avx2.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/conda/lib/libmkl_core.so' from LD_PRE

In [ ]:
# ============================================
# Fine-Tune SBERT on Tweet ↔ Paper Pairs
# ============================================

# Load train data and paper collection
train_df = pd.read_csv("subtask4b_query_tweets_train.tsv", sep="\t", names=["post_id", "tweet_text", "cord_uid"])
with open("subtask4b_collection_data.pkl", "rb") as f:
    papers_df = pickle.load(f)

# Merge paper title + abstract
papers_df["text"] = papers_df["title"] + ". " + papers_df["abstract"]
paper_dict = dict(zip(papers_df["cord_uid"], papers_df["text"]))

# Prepare training examples
train_samples = []
for _, row in train_df.iterrows():
    if row["cord_uid"] in paper_dict:
        tweet = row["tweet_text"]
        paper = paper_dict[row["cord_uid"]]
        train_samples.append(InputExample(texts=[tweet, paper]))

# Load model and fine-tune
model = SentenceTransformer("multi-qa-mpnet-base-cos-v1")

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
print("Starting training now...")
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=6,
    warmup_steps=100,
    show_progress_bar=True
)

# Save model after training
model.save("fine_tuned_sbert")

Starting training now...


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]

Iteration:   0%|          | 0/804 [00:00<?, ?it/s]